In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


import numpy as np
import random
import os
import Pyclass 



from torch.utils.data import Dataset, DataLoader

# PYCLASS IMPORT

In [2]:
sound_train = Pyclass.my_datset2(part='new_train',option='spectrum_Mel_PCA',train = True)
                         

sound_test = Pyclass.my_datset2(part='new_test',option='spectrum_Mel_PCA',train = False)

In [51]:
batch_size = 10

train_loader  = DataLoader(dataset=sound_train,
                                           batch_size=batch_size,
                                           shuffle=False
                                           ,num_workers=1)

test_loader = DataLoader(dataset=sound_test,
                                         batch_size=batch_size,
                                         shuffle=False,
                        num_workers=1)

In [52]:
print(sound_train.train_data.shape)
print(sound_train.train_label.shape)
print(sound_train.train_data.dtype)

torch.Size([452, 1, 40, 40])
torch.Size([452])
torch.float32


# DEFINE MODEL

In [43]:
class CNN(nn.Module):
    def __init__(self):
        
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 1,out_channels=16,kernel_size = 5),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size = 5),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(in_channels = 32,out_channels = 64,kernel_size = 5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            
        )
        
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*6*6,100),
            nn.ReLU(),
            nn.Linear(100,8),
            nn.Softmax()
        )       
        
    def forward(self,x):
        print(x.data.shape)
        out = self.layer(x)
        out = out.view(out.shape[0],-1)
        out = self.fc_layer(out)

        return out

In [44]:
model = CNN().cuda()

In [45]:
loss = nn.CrossEntropyLoss()

optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# TRAIN MODEL

In [46]:
num_epochs = 300

In [47]:
save_path = '/home/libedev/mute/mute-hero/download/dataset/model/'
model_path = save_path + 'model.pkl'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [48]:
if os.path.isfile(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Model Loaded!")

else:
    
    for epoch in range(num_epochs):

        total_batch = len(sound_train) // batch_size

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            X = batch_images.cuda()
            Y = batch_labels.cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 10 == 0:
                print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                     %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

    if not os.path.isfile(model_path):
        print("Model Saved!")
        torch.save(model.state_dict(), model_path)

Model Loaded!


# Transfer Learning

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 8)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# TEST MODEL

In [39]:
model.eval()

correct = 0
total = 0

for sound, labels in test_loader:
    
    sound = sound.cuda()
    outputs = model(sound)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test sound: %f %%' % (100 * float(correct) / total))

torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([10, 1, 40, 40])
torch.Size([8, 1, 40, 40])
Accuracy of test sound: 53.571429 %
